In [1]:
import sys

# user module imports
sys.path.append("../")
from src.common.file_io import coord_to_json, load_tiff_images
from src.utils.image_utils import image_crop_walk

## Loading in image data

TIFF Images are loaded form the `./sample_data` folder by using the `load_tiff_images()` function. The `load_tiff_images()` returns a python generator that contains `PIL` Image objects.

Since these are python generators, all the images will not be loaded into memory.


In [2]:
# load image data from directory
loaded_imgs = load_tiff_images(img_dir="./sample_data")

In [3]:
# getting all cropped images meta data into
# -- here we want images with a size of 256 by 256
cropped_images_list = []
for image_obj in loaded_imgs:
    cropped_images = image_crop_walk(image_obj, 256, 256)
    cropped_images_list.append(cropped_images)

## Writing out ImageCropSelection Objects into JSON file

Now that we have a list of metadata associated with each crop of the source image, we can write it out into a JSON file, which will be used for our GUI to read and select.

Here we are using a function called `coord_to_json` that can be found in the `src/common/_io.py` module. Writes out the array of `ImageCropSelection` objects into JSON format. The function will return the `Path` object where the `JSON` file was saved.


In [4]:
json_path = coord_to_json(cropped_images_list, outname="test_image_coords")
json_path

PosixPath('/home/erikserrano/Development/macrophage-striation-analysis/notebooks/test_image_coords.json')